In [189]:
import pandas as pd
import spacy
import re
import string

from spacy.tokens import Token
from tqdm import tqdm

# from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from textblob import TextBlob
from textblob import Word

In [3]:
df = pd.read_csv("..//data//Womens Clothing E-Commerce Reviews.csv")
# Removed index from dataframe
df=df.drop(columns=["Unnamed: 0"])

In [174]:
df['Review Text'].head(67314).tail().tolist()

["I was very happy to snag this dress at such a great price! it's very easy to slip on and has a very flattering cut and color combo.",
 'It reminds me of maternity clothes. soft, stretchy, shiny material. cut is flattering and drapes nicely. i only found one button to close front... looked awkward. nice long sleeves.\nnot for me but maybe for others. just ok.',
 "This fit well, but the top was very see through. this never would have worked for me. i'm glad i was able to try it on in the store and didn't order it online. with different fabric, it would have been great.",
 "I bought this dress for a wedding i have this summer, and it's so cute. unfortunately the fit isn't perfect. the medium fits my waist perfectly, but was way too long and too big in the bust and shoulders. if i wanted to spend the money, i could get it tailored, but i just felt like it might not be worth it. side note - this dress was delivered to me with a nordstrom tag on it and i found it much cheaper there after l

In [172]:
df['Review Text'].head(50).tail().str.replace("\\'","'").tolist()

['I bought this item from online... the fit on the model looked a little loose but when i got mine it seemed a bit tight! so i took it back to the store & ordered a larger size. for the sale price this is a great top.',
 "I love this top. i wear it all the time.  the problem is that you can tell i wear it all the time as the fabric has started to fade.  i'd still recommend it as it is so comfortable.",
 'Very comfortable, material is good, cut out on sleeves flattering',
 "This sweater is perfect for fall...it's roomy, warm, super comfy and the color really pops.",
 "Really cute top! the embroidery on the collar & bib of this top is beautiful and unique, received lots of compliments & questions about where i got it. i'm rather busty up top and the medium fit well, a small probably would've fit better but it's still super adorable."]

\'


In [4]:
def renames(feature, values):
    Dict_cols={}
    for value in values:
        Dict_cols[value]=feature+value
    return Dict_cols

In [5]:
pd.isnull(df['Review Text']).value_counts()

False    22641
True       845
Name: Review Text, dtype: int64

In [6]:
pd.isnull(df['Title']).value_counts()

False    19676
True      3810
Name: Title, dtype: int64

In [7]:
column_name='Division Name'
dummies= pd.get_dummies(df[column_name])
Encoded_Data=pd.concat([df,dummies], axis=1,ignore_index=False)
Encoded_Data=Encoded_Data.rename(index=str, columns=renames(column_name+'_',dummies.columns.tolist()))

column_name='Department Name'
dummies= pd.get_dummies(Encoded_Data[column_name])
Encoded_Data=pd.concat([Encoded_Data,dummies], axis=1,ignore_index=False)
Encoded_Data=Encoded_Data.rename(index=str, columns=renames(column_name+'_',dummies.columns.tolist()))

column_name='Class Name'
dummies= pd.get_dummies(Encoded_Data[column_name])
Encoded_Data=pd.concat([Encoded_Data,dummies], axis=1,ignore_index=False)
Encoded_Data=Encoded_Data.rename(index=str, columns=renames(column_name+'_',dummies.columns.tolist()))

In [8]:
Encoded_Data.columns

Index(['Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Department Name', 'Class Name', 'Division Name_General',
       'Division Name_General Petite', 'Division Name_Initmates',
       'Department Name_Bottoms', 'Department Name_Dresses',
       'Department Name_Intimate', 'Department Name_Jackets',
       'Department Name_Tops', 'Department Name_Trend', 'Class Name_Blouses',
       'Class Name_Casual bottoms', 'Class Name_Chemises',
       'Class Name_Dresses', 'Class Name_Fine gauge', 'Class Name_Intimates',
       'Class Name_Jackets', 'Class Name_Jeans', 'Class Name_Knits',
       'Class Name_Layering', 'Class Name_Legwear', 'Class Name_Lounge',
       'Class Name_Outerwear', 'Class Name_Pants', 'Class Name_Shorts',
       'Class Name_Skirts', 'Class Name_Sleep', 'Class Name_Sweaters',
       'Class Name_Swim', 'Class Name_Trend'],
      dtype='object')

In [193]:
'asdd'.translate(str.maketrans('','',string.punctuation))

'asdd'

In [210]:
nlp = spacy.load("en_core_web_sm")

def remove_spell_errors(doc):
    bdoc = TextBlob(str(doc))
    ## Correcting the words
    return str(bdoc.correct())

def stop(doc):
    #return [token for token in doc if not token.is_punct and not token.text.isalnum() and not token.is_digit and not token.is_stop]
    #return [token for token in doc if token.text.isalnum() and not token.is_digit and not token.is_stop]
    return [token for token in doc if not token.is_digit and not token.is_stop and ( token.text.isalpha() or not token.text.isalnum())]

def lemmatize(doc):
    return [token.lemma_.lower() if token.lemma_ != "-PRON-" else token.text.lower() for token in doc]

def remove_line_breaks(doc):
    return [token.replace("\n", " ").replace("\r", " ") for token in doc]

nlp.add_pipe(stop)
nlp.add_pipe(lemmatize)
nlp.add_pipe(remove_line_breaks)

In [237]:
docs = Encoded_Data["Review Text"].apply(remove_spell_errors).to_list()
processed_docs = []

with tqdm(total=len(docs)) as bar:
    for doc in nlp.pipe(docs):
            line = " ".join(doc)
            ## Removing the punctuation
            line=line.translate(str.maketrans('','',string.punctuation))
            ## Removing numbers 
            line=" ".join(list(filter(lambda w : not w.isdigit(), line.split())))
            processed_docs.append(line)
            bar.update(1)

df["processed_Review_text"] = processed_docs

KeyboardInterrupt: 

In [ ]:
df["Review Text"].tolist()[30:40]

In [ ]:
df["processed_Review_text"].tolist()[30:40]

In [ ]:
vectorizer = TfidfVectorizer()
review_vectors = vectorizer.fit_transform(df["processed_Review_text"])
features_df = pd.DataFrame(review_vectors.toarray(), columns = vectorizer.get_feature_names())

In [ ]:
Prep_data=pd.concat([Encoded_Data,features_df], axis=1,ignore_index=False)
Prep_data=Prep_data.drop(columns=['Clothing ID','Title','Division Name', 'Department Name', 'Class Name'])
#Prep_data=Prep_data.drop(columns=Prep_data.columns.tolist()[33:176]) 

In [ ]:
Prep_data.columns.tolist()[1:176]